In [ ]:
import pandas as pd
import requests

In [ ]:
convirt_query_df = pd.read_csv("/opt/gpudata/imadejski/search-model/ctds-search-model/data/convirt_queries.csv")

In [ ]:
print(convirt_query_df)

In [ ]:
convirt_labels = [
    "Pneumothorax",
    "Pneumonia",
    "Fracture",
    "Cardiomegaly",
    "Pleural Effusion",
    "Edema",
    "Atelectasis",
    "No finding"
]

In [ ]:
def make_query_expansions(labels):
    entries = []
    for label in labels:
        entries.extend([
            {"Variable": label, "Text": f"Findings consistent with {label}"},
            {"Variable": label, "Text": f"Findings suggesting {label}"},
            {"Variable": label, "Text": f"Findings are most compatible with {label}"},
            {"Variable": label, "Text": f"{label} seen"},
        ])

    df = pd.DataFrame(entries)
    return df

In [ ]:
curr_queries_df = make_query_expansions(convirt_labels)

In [ ]:
print(curr_queries_df)

In [ ]:
def query_embedding_from_df(df):
    for row in df:
        row["Text"] = query
        